# FinGPT Test: News with GPT Instructions (NWGI)

This notebook demonstrates how to test FinGPT on the News with GPT Instructions sentiment analysis dataset.

## 1. Install Dependencies

In [ ]:
!pip install transformers==4.32.0 peft==0.5.0 datasets accelerate bitsandbytes sentencepiece tqdm scikit-learn pandas matplotlib seaborn

## 2. Clone the FinGPT Repository

In [ ]:
!git clone https://github.com/AI4Finance-Foundation/FinGPT.git
%cd FinGPT

## 3. Download the NWGI Dataset

In [ ]:
import datasets
from pathlib import Path
data_dir = Path('./fingpt/FinGPT_Benchmark/data')
data_dir.mkdir(parents=True, exist_ok=True)

print("Downloading News with GPT Instructions dataset...")
dataset = datasets.load_dataset('oliverwang15/news_with_gpt_instructions')

# Save the dataset to disk
save_path = str(data_dir / "news_with_gpt_instructions")
print(f"Saving dataset to {save_path}")
dataset.save_to_disk(save_path)
print("Dataset download complete!")

## 4. Testing Module for NWGI

In [ ]:
%%writefile fingpt/FinGPT_Benchmark/benchmarks/nwgi.py
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
import datasets
import torch
from pathlib import Path

dic = {
    'strong negative':"negative",
    'moderately negative':"negative",
    'mildly negative':"neutral",
    'strong positive':"positive",
    'moderately positive':"positive",
    'mildly positive':'neutral',
    'neutral':'neutral',
}

def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

def change_target(x):
    if 'positive' in x or 'Positive' in x:
        return 'positive'
    elif 'negative' in x or 'Negative' in x:
        return 'negative'
    else:
        return 'neutral'

def test_nwgi(args, model, tokenizer, prompt_fun=None):
    batch_size = args.batch_size
    # dataset = load_dataset('oliverwang15/news_with_gpt_instructions')
    dataset = load_from_disk(Path(__file__).parent.parent / 'data/news_with_gpt_instructions/')
    dataset = dataset['train']
    dataset = dataset.to_pandas()
    dataset['output'] = dataset['label'].apply(lambda x:dic[x])

    if prompt_fun is None:
        dataset["instruction"] = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."
    else:
        dataset["instruction"] = dataset.apply(prompt_fun, axis = 1)
    dataset["input"] = dataset["news"]

    dataset = dataset[['input', 'output', 'instruction']]
    dataset[["context","target"]] = dataset.apply(format_example, axis = 1, result_type="expand")

    print(f"\n\nPrompt example:\n{dataset['context'][0]}\n\n")

    context = dataset['context'].tolist()

    total_steps = dataset.shape[0]//batch_size + 1
    print(f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text_list = []
    for i in tqdm(range(total_steps)):
        tmp_context = context[i* batch_size:(i+1)* batch_size]
        if not tmp_context:
            continue
        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True, max_length=512, return_token_type_ids=False)
        for k in tokens.keys():
            tokens[k] = tokens[k].cuda()
        res = model.generate(**tokens, max_length=512, eos_token_id=tokenizer.eos_token_id)
        res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]
        if i % 20 == 0:
            tqdm.write(f'Example {i}: {res_sentences[0]}')
        out_text = [o.split("Answer: ")[1] if "Answer: " in o else o for o in res_sentences]
        out_text_list += out_text
        torch.cuda.empty_cache()

    dataset["out_text"] = out_text_list
    dataset["new_target"] = dataset["target"].apply(change_target)
    dataset["new_out"] = dataset["out_text"].apply(change_target)

    acc = accuracy_score(dataset["new_target"], dataset["new_out"])
    f1_macro = f1_score(dataset["new_target"], dataset["new_out"], average = "macro")
    f1_micro = f1_score(dataset["new_target"], dataset["new_out"], average = "micro")
    f1_weighted = f1_score(dataset["new_target"], dataset["new_out"], average = "weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    # Save results to CSV
    dataset.to_csv('nwgi_results.csv', index=False)
    print("Results saved to nwgi_results.csv")

    return dataset

## 5. Update Benchmarking Script

In [ ]:
%%writefile fingpt/FinGPT_Benchmark/benchmarks/benchmarks.py
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, get_peft_model, LoraConfig, TaskType  # 0.4.0
import torch
import argparse

from nwgi import test_nwgi

import sys
sys.path.append('../')
from utils import *

def main(args):
    if args.from_remote:
        model_name = parse_model_name(args.base_model, args.from_remote)
    else:
        model_name = '../' + parse_model_name(args.base_model)

    model = AutoModelForCausalLM.from_pretrained(
        model_name, trust_remote_code=True,
        # load_in_8bit=True
        device_map="auto",
        # fp16=True
    )
    model.model_parallel = True

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    # tokenizer.pad_token_id = tokenizer.eos_token_id

    tokenizer.padding_side = "left"
    if args.base_model == 'qwen':
        tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('<|endoftext|>')
        tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('<|extra_0|>')
    if not tokenizer.pad_token or tokenizer.pad_token_id == tokenizer.eos_token_id:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

    print(f'pad: {tokenizer.pad_token_id}, eos: {tokenizer.eos_token_id}')

    model = PeftModel.from_pretrained(model, args.peft_model)
    model = model.eval()

    with torch.no_grad():
        for data in args.dataset.split(','):
            if data == 'nwgi':
                test_nwgi(args, model, tokenizer)
            else:
                raise ValueError('undefined dataset.')

    print('Evaluation Ends.')


if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", required=True, type=str)
    parser.add_argument("--base_model", required=True, type=str, choices=['chatglm2', 'llama2', 'llama2-13b', 'llama2-13b-nr', 'baichuan', 'falcon', 'internlm', 'qwen', 'mpt', 'bloom'])
    parser.add_argument("--peft_model", required=True, type=str)
    parser.add_argument("--max_length", default=512, type=int)
    parser.add_argument("--batch_size", default=4, type=int, help="The train batch size per device")
    parser.add_argument("--instruct_template", default='default')
    parser.add_argument("--from_remote", default=False, type=bool)

    args = parser.parse_args()

    print(args.base_model)
    print(args.peft_model)

    main(args)

## 6. Create Utils Module

In [ ]:
%%writefile fingpt/FinGPT_Benchmark/utils.py
def parse_model_name(base_model, from_remote=False):
    model_map = {
        'chatglm2': 'THUDM/chatglm2-6b',
        'llama2': 'meta-llama/Llama-2-7b-hf',
        'llama2-13b': 'meta-llama/Llama-2-13b-hf',
        'llama2-13b-nr': 'NousResearch/Llama-2-13b-hf',
        'baichuan': 'baichuan-inc/Baichuan-7B',
        'falcon': 'tiiuae/falcon-7b',
        'internlm': 'internlm/internlm-7b',
        'qwen': 'Qwen/Qwen-7B',
        'mpt': 'mosaicml/mpt-7b',
        'bloom': 'bigscience/bloom-7b1',
    }
    if base_model not in model_map:
        raise ValueError(f"Unknown base model: {base_model}")
    return model_map[base_model]

In [ ]:
from huggingface_hub import login
login(token="token")

## 7. Run the NWGI Benchmark Test

In [ ]:
%cd /content/FinGPT/fingpt/FinGPT_Benchmark/benchmarks

base_model = 'llama2'
# The FinGPT adapter model
peft_model = 'FinGPT/fingpt-mt_llama2-7b_lora'
batch_size = 4
max_length = 512

!python benchmarks.py --dataset nwgi --base_model {base_model} --peft_model {peft_model} --batch_size {batch_size} --max_length {max_length} --from_remote True

Example 3520: Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Focus on the business, not the stock split.
Answer:  neutral
Example 3540: Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Long-term investors looking for top-notch growth stocks may want to consider these seven world-class companies as core holdings right now. The post 7 Great Growth Stocks to Add to Your January Buy List appeared first on InvestorPlace.
Answer:  positive
Example 3560: Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Texas Instruments reported solid Q2 results on Tuesday, beating analyst expectations by a significant margin. Cash flow decreased substantially compared to the same period last year. There is an explanation for that.
Answer:  negative
Example 3580: Instruction: What is the sentiment of this news? Plea